In [11]:
!pip install wandb
!wandb login

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 13.9 MB/s 
     |████████████████████████████████| 168 kB 66.8 MB/s 
     |████████████████████████████████| 182 kB 71.3 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 73.8 MB/s 
     |████████████████████████████████| 166 kB 69.2 MB/s 
     |████████████████████████████████| 162 kB 68.9 MB/s 
     |████████████████████████████████| 162 kB 70.6 MB/s 
     |████████████████████████████████| 158 kB 77.9 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 68.1 MB/s 
     |████████████████████████████████| 157 kB 70.6 MB/s 
     |████████████████████████████████| 157 kB 71.8 MB/s 
     |████████████████████████████████| 157 kB 71.5 MB/s 
     |████████████████████████████████| 157 kB 77.1 MB/s 
     |███████████████████████████

In [3]:
!pip install !pip install tensorflow_io
!pip install keras-tuner
!pip install print_schema
!pip install pydub
!pip install opensmile
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers

ERROR: Invalid requirement: '!pip'
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
import wandb
wandb.init(project="speech-fall-project")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kaanyarali. Use `wandb login --relogin` to force relogin


In [4]:
import torch
import torchvision
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

/usr/local/lib/python3.7/dist-packages/torch/hub.py:267: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  "You are about to download and run code from an untrusted repository. In a future release, this won't "
Downloading: "https://github.com/huggingface/pytorch-transformers/zipball/main" to /root/.cache/torch/hub/main.zip


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
import itertools
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

hiddenLayerDims = [64,128,256,512]
pset = list(powerset(hiddenLayerDims))
archs = []
for i in range(1,len(pset)):
    res = list(itertools.permutations(pset[i]))
    for j in res:
      archs.append(j)

In [8]:
sweep_config = {
    'method': 'grid'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric
parameters_dict = {
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    'archs': {
        'values': archs
        }  
    }

sweep_config['parameters'] = parameters_dict
parameters_dict.update({
    'epochs': {
        'value': 3}
    })
parameters_dict.update({
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'value': 16
      }
    })

In [9]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'archs': {'values': [(64,),
                                     (128,),
                                     (256,),
                                     (512,),
                                     (64, 128),
                                     (128, 64),
                                     (64, 256),
                                     (256, 64),
                                     (64, 512),
                                     (512, 64),
                                     (128, 256),
                                     (256, 128),
                                     (128, 512),
                                     (512, 128),
                                     (256, 512),
                                     (512, 256),
                                     (64, 128, 256),
                                     (64, 256, 128),
                                     (128, 64, 256),
            

In [14]:
sweep_id = wandb.sweep(sweep_config, project="speech-fall-project")

Create sweep with ID: 9a72v36a
Sweep URL: https://wandb.ai/kaanyarali/speech-fall-project/sweeps/9a72v36a


In [34]:
import pandas as pd

data = pd.read_csv("train.csv") #load training csv file
data["Content"].fillna("",inplace=True)
train_texts = list(data.loc[:,"Content"])
train_labels = list(data.loc[:,"Label"])

val_texts = train_texts[:33]
val_labels = train_labels[:33]
train_texts = train_texts[33:]
train_labels = train_labels[33:]


df = pd.read_csv("test.csv") #load test csv file
df["Content"].fillna("",inplace=True)
test_texts = list(df.loc[:,"Content"])
test_labels = list(df.loc[:,"Label"])

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [35]:
class ADdataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [36]:
train_dataset = ADdataset(train_encodings, train_labels)
val_dataset = ADdataset(val_encodings, val_labels)

In [37]:
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, archs,dropout,freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 256, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        
        layers = []          
        layers.append(nn.Linear(D_in, archs[0]))
        layers.append(nn.BatchNorm1d(num_features=archs[0]))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
          
        for i in range(1,len(archs)):
          layers.append(nn.Linear(archs[i-1], archs[i]))
          layers.append(nn.BatchNorm1d(num_features=archs[i]))
          layers.append(nn.ReLU())
          layers.append(nn.Dropout(dropout))

        layers.append(nn.Linear(archs[-1], 1))
        self.classifier  = nn.Sequential(*layers)

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

In [38]:
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader


def initialize_model(archs,dropout,train_loader,epochs=50):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(archs,dropout,freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_loader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [40]:
import random
import time

# Specify loss function
#loss_fn = nn.CrossEntropyLoss()
#loss_fn = nn.BCELoss()
loss_fn =  torch.nn.BCEWithLogitsLoss()
def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train_epoch(model, train_loader,val_loader, optim,scheduler):
      model.train()
      # For each batch of training data...
      train_loss_sum = 0
      train_accuracy_epoch = 0
      for batch in (train_loader):
          optim.zero_grad()
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['labels'].to(device)

          # Perform a forward pass. This will return logits.
          logits = model(input_ids, attention_mask)

          # Compute loss and accumulate the loss values
          logits = logits.reshape(-1) #silebilirsin
          
          loss = loss_fn(logits, labels.float())
          train_loss_sum += loss.item()

          logits_class = logits > 0.5

          train_acc = (labels == logits_class).sum().item() / labels.size(0)
          train_accuracy_epoch += train_acc

          # Perform a backward pass to calculate gradients
          loss.backward()
          # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
          # Update parameters and the learning rate
          optim.step()
          scheduler.step()
          #wandb.log({"batch loss": loss.item()})

      model.eval()
      # Tracking variables
      val_loss_sum = 0
      val_accuracy_epoch = 0
      # For each batch in our validation set...
      for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Compute logits
        with torch.no_grad():
          logits = model(input_ids, attention_mask)

          # Compute loss
          logits = logits.reshape(-1)
          loss = loss_fn(logits, labels.float())
          val_loss_sum +=loss.item()

          logits_class = logits > 0.5
          val_acc = (labels == logits_class).sum().item() / labels.size(0)
          val_accuracy_epoch += val_acc

      return (train_loss_sum / len(train_loader)),(train_accuracy_epoch/len(train_loader)),(val_loss_sum / len(val_loader)),(val_accuracy_epoch/len(val_loader))

def train(model, train_loader, optim, val_loader=None, epochs=50, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    train_loss_list = []
    val_loss_list = []
    train_acc_list = []
    val_acc_list = [] 
    for epoch_i in range(epochs):
        train_loss_sum = 0
        train_accuracy_epoch = 0
        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for batch in (train_loader):
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            # Perform a forward pass. This will return logits.
            logits = model(input_ids, attention_mask)

            # Compute loss and accumulate the loss values
            logits = logits.reshape(-1) #silebilirsin
            
            loss = loss_fn(logits, labels.float())
            train_loss_sum += loss.item()

            logits_class = logits > 0.5
            train_acc = (labels == logits_class).sum().item() / labels.size(0)
            train_accuracy_epoch += train_acc

            # Perform a backward pass to calculate gradients
            loss.backward()


            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

        avg_train_loss = np.round(train_loss_sum/len(train_loader),2)
        avg_train_acc = np.round(train_accuracy_epoch/len(train_loader),2)

        if evaluation == True:
            avg_val_loss, avg_val_acc = evaluate(model, val_loader)
        print('Epoch {}, train loss {} , val loss is {}, train acc is {}, val acc is {} '.format(epoch_i,avg_train_loss,avg_val_loss,avg_train_acc,avg_val_acc))
        train_loss_list.append(avg_train_loss)
        val_loss_list.append(avg_val_loss)
        val_acc_list.append(avg_val_acc)
        train_acc_list.append(avg_train_acc)

    print("Training complete!")
    return model,train_loss_list,val_loss_list,train_acc_list,val_acc_list

def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_loss_sum = 0
    val_accuracy_epoch = 0
    # For each batch in our validation set...
    for batch in val_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        val_loss_sum +=loss.item()
        avg_val_loss = np.round(val_loss_sum/len(val_dataloader),2)

        logits_class = logits > 0.5
        val_acc = (labels == logits_class).sum().item() / labels.size(0)
        val_accuracy_epoch += val_acc
        avg_val_acc = np.round(val_accuracy_epoch/len(val_dataloader),2)

    return avg_val_loss, avg_val_acc

def evaluate_test(model, test_dataloader):
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    test_loss_sum = 0
    test_accuracy_epoch = 0
    predictions = []
    labels_list = []
    for batch in test_dataloader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      
      # Compute logits
      with torch.no_grad():
        logits = model(input_ids, attention_mask)
        labels_list.append(labels.cpu())

        # Compute loss
        logits = logits.reshape(-1)
        loss = loss_fn(logits, labels.float())
        test_loss_sum +=loss.item()
        avg_test_loss = np.round(test_loss_sum/len(test_dataloader),5)

        logits_class = logits > 0.5
        predictions.append(logits.cpu())
        test_acc = (labels == logits_class).sum().item() / labels.size(0)
        test_accuracy_epoch += test_acc
        avg_test_acc = np.round(test_accuracy_epoch/len(test_dataloader),5)

    return avg_test_loss, avg_test_acc, predictions, labels_list

In [41]:
def train_wb(config=None):
    # Initialize a new wandb run
    #run = wandb.init(config=config,name="dr-{},arch-{},bs-{}".format(config.dropout,config.archs,config.batch_size))
    #with run:
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        wandb.run.name = "dr-{},arch-{},bs-{}".format(config.dropout,config.archs,config.batch_size)
        wandb.run.save()    

        train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=True)
        bert_classifier, optimizer, scheduler = initialize_model(config.archs,config.dropout,train_loader,epochs=3)

        for epoch in range(config.epochs):
            train_loss,train_acc,val_loss,val_acc = train_epoch(bert_classifier, train_loader,val_loader, optimizer,scheduler)
            #wandb.log({"loss": avg_loss, "epoch": epoch})  
            wandb.log({"Training Loss": train_loss,"Training Acc": train_acc,"Validation Loss": val_loss,"Validation Acc": val_acc})

In [ ]:
set_seed(42)    # Set seed for reproducibility
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
#wandb.agent(sweep_id, train_wb, count=3)
wandb.agent(sweep_id, train_wb)